# Sensitivity Analysis and Data Assimilation on the example of Lorenz and SEIR models

Today's tutorial will be divided into three parts:

1. Introduction to Lorenz and SEIR models
1. Sensitivity analysis
1. Data assimilation with 4DVar

# Introduction to Lorenz and SEIR models



### Lorenz

![Lorenz equations](images/lorenz.png)

source: https://en.wikipedia.org/wiki/Lorenz_system

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D

Method that computes one timestep of Lorenz model:

In [ ]:
def lorenz_step(state, _, rho, sigma, beta):
    x, y, z = state  # Unpack the state vector
    return sigma * (y - x), x * (rho - z) - y, x * y - beta * z  # Derivatives

Let's draw the model's trajectory for the basic parameters:

In [ ]:
rho = 28.0
sigma = 10.0
beta = 8.0 / 3.0

state0 = [1,1,1]

t = np.arange(0.0, 40.0, 0.01)

states = odeint(lorenz_step, state0, t, args=(rho,sigma,beta))

fig = plt.figure()
ax = fig.gca(projection="3d")
ax.plot(states[:, 0], states[:, 1], states[:, 2])
plt.draw()
plt.show()

### SEIR

![SEIR epidemic model](images/seir.png)

SEIR is a model that simulates an outbreak of a virus.

The population is divided into four groups:
* S - group of humans **susceptible** to be exposed to the virus. Before an epidemic starts, all population members belong to this group.
* E - group of people that were **exposed** to the virus. In this stage they already started to develop the illness, but they don't show symptoms nor infect others yet.
* I - group of **infectious** individuals, who can pass the virus to susceptible members.
* R - people that have already **recovered** from the infection.

Additionaly we can specify two more groups:
* D - a group of individuals that are **dead** as a result of the infection
* N - total number of alive individuals

---

There are four parameters. The reasonable bounds for them are given in the brackets:
* alfa - the case fatality rate - fraction of Infected group that dies each day. [0.001, 0.01]
* beta - probability of disease transmission times the number of contacts per day. [0, 7]
* epsilon - rate of progression from Exposed to Infectious (so 1/eps is the incubation period). [0.05, 0.5] (from 2 to 20 days)
* gamma - rate of progression from Infectious to Recovered (so 1/gamma is the length of the infectious period) [0.05, 0.5] (from 2 to 20 days)


---

These are the equations which represent the group size in the following time step:

S = S - S \* beta \* I / N  
E = E + S \* beta \* I / N - eps \* E  
I = I + eps \* E - (alfa + gamma) \* I  
R = R + gamma \* I  
D = alfa \* I  


In [ ]:
def seir_step(state):
    state = list(np.ravel(state))
    state = list(map(lambda x : max(x, 0), state))
    S_old, E_old, I_old, R_old, D_old, alfa, beta, eps, gamma = state

    N = S_old + E_old + I_old + R_old
    D = D_old + alfa * I_old
    S = S_old - S_old * beta * I_old / N
    E = E_old + S_old * beta * I_old / N - eps * E_old
    I = I_old + eps * E_old - (alfa + gamma) * I_old
    R = R_old + gamma * I_old

    results = [S, E, I, R, D, alfa, beta, eps, gamma]
    return np.array(results).reshape(-1, 1)

Let's simulate a sample epidemic

**TASK 1** Choose some values from the reasonable bounds introduced earlier and observe the effect on the plots



In [ ]:
# TODO: insert values for alfa, beta, eps, gamma
params = [ ??? ]
seird = [40000000, 1, 0, 0, 0]
state = np.array(seird + params).reshape(-1, 1)

days = 365

states = np.empty((days,5))
for i in range(days):
    states[i]  = state[:5][:,0]
    state = seir_step(state)

plt.plot( range(days),states[:,0])
plt.plot( range(days),states[:,1])
plt.plot( range(days),states[:,2])
plt.plot( range(days),states[:,3])
plt.plot( range(days),states[:,4])
print('Susceptible, exposed, infectious, recovered and dead individuals')

In [ ]:
plt.plot( range(days),states[:,1])
plt.plot( range(days),states[:,2])
plt.plot( range(days),states[:,4])
print('Exposed, infectious and dead individuals')

# Sensitivity analysis

In order to check which parameters of the model have the biggest influence on the output, we can perform a Sensitivity Analysis. SALib is a Python library that implements it. It does two things:


1.   Generates a set of inputs to be fed to the model
2.   Analyses outputs returned by the model for this set of inputs


In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol

### Sensitivity analysis - Lorenz 63 model

We will perform SA on Lorenz 63 model. First, we need to define the parameters of the model and generate a set of input parameters.

In [ ]:
problem = {
    'num_vars': 3,
    'names': ['rho', 'sigma', 'beta'],
    'bounds': [[14.0,42.0],
               [3.0, 20.0],
               [7.0/3.0, 9.0/3.0]]
}

#Let's generate 20 values for each parameter:
input = saltelli.sample(problem, 20)

print(input.shape)
print(input[:5])

Now, we need to define our model and generate an output value for each input (each set of parameters)

Lorenz model has a 3-dimensional state (x,y,z) for each timestep. However, SA can only analyse a single output for a given input.

In order to deal with that, we will analyse the sensitivity in the 3 dimensions separately. We will analyse the standard deviation of values in each dimension.

In [ ]:
def generate_outputs_lorenz(parameter_combinations):
    # We need an output array with a row for each generated input and 3 colums for 3 output variables (x, y, z)
    output = np.empty( shape=(parameter_combinations.shape[0],3) )
    # Define timesteps for Lorenz 63 integration
    t = np.arange(0.0, 40.0, 0.01)

    for i, params in enumerate(parameter_combinations):
        initial_state = [1,1,1]
        rho, sigma, beta = params[0], params[1], params[2]

        # Get states in each timestep:
        states = odeint(lorenz_step, initial_state, t, args=(rho, sigma, beta))

        # We take the standard deviation in each dimension as the output.
        output[i] = np.std(states, axis=0)
    return output

output = generate_outputs_lorenz(input)

Finally, we can analyse the sensitivity. We analyse it in respect to each output dimension separately

In [ ]:
Si_X = sobol.analyze(problem, output[:,0], print_to_console=False)['S1']
Si_Y = sobol.analyze(problem, output[:,1], print_to_console=False)['S1']
Si_Z = sobol.analyze(problem, output[:,2], print_to_console=False)['S1']

print('           ', list(map(lambda name: name + ' '*(5-len(name)),    problem['names'])))
print('X analysis:', list(map(lambda number: '{0:.3f}'.format(number),  Si_X)))
print('Y analysis:', list(map(lambda number: '{0:.3f}'.format(number),  Si_Y)))
print('Z analysis:', list(map(lambda number: '{0:.3f}'.format(number),  Si_Z)))

The higher the value, the more the parameter affects the behaviour of the model.
The rho parameter influences all dimensions of model significantly.

### Sensitivity analysis - SEIR model

Now we will perform a similar SA on the SEIR model

**TASK 2** Just like in the Lorenz example, define the number of variables, their names and the bounds. Use the bounds provided in the definition of the model

In [ ]:
problem_seir = {
    'num_vars': ??? ,
    'names': ??? ,
    'bounds': ???
}

input = saltelli.sample(problem_seir, 100)

print(input.shape)
print(input[:5])

Again we encounter the same problem: SA can analyse only one 1-dimensional output for a given input, while SEIR is a time series with 5 groups (5 dimensions). We will deal with that in two ways separately:
* analyse the final amount of dead individuals
* analyse the maximum amount of infected individuals at a single moment

In [ ]:
def generate_outputs_seir(parameter_combinations):
    # We need an output array with a row for each generated input
    output_final = np.empty( shape=(parameter_combinations.shape[0]) )
    output_max = np.empty( shape=(parameter_combinations.shape[0]) )

    for i, params in enumerate(parameter_combinations):
        initial_state = np.array([39900000, 60000, 10000, 80000, 2300])
        state = np.concatenate((initial_state, params))

        max_infected = 0
        for step in range(200):
            state = seir_step(state)
            if state[:][2] > max_infected:
                max_infected = state[:][2]
        output_final[i] = state[:][4]
        output_max[i] = max_infected
    return output_final, output_max

output_final, output_max = generate_outputs_seir(input)

In [ ]:
print('                      ', list(map(lambda name: name + ' '*(5-len(name)),    problem_seir['names'])))
Si = sobol.analyze(problem_seir, output_final, print_to_console=False)['S1']
print('Final deaths analysis:', list(map(lambda number: '{0:.3f}'.format(number),  Si)))
Si = sobol.analyze(problem_seir, output_max, print_to_console=False)['S1']
print('Max infected analysis:', list(map(lambda number: '{0:.3f}'.format(number),  Si)))

# Data assimilation with 4DVar

This algorithm realizes an estimation of the state of a dynamic system, by a variational minimization method of the classical J function in data assimilation:
![4dvar function](images/4dvar.png)

For calculations we used ADAO python library - [documentation](https://docs.salome-platform.org/latest/gui/ADAO/en/ref_algorithm_4DVAR.html)

### 4d var with Lorenz 63 model

First we will try to find parameters of Lorenz63 model using 4d-var data assimilation

In [ ]:
from data_assimilation import assimilate

In [ ]:
rho = 28.0
sigma = 10.0
beta = 8.0 / 3.0

d = 0.01
xb = [1., 1., 1.] + [rho, sigma, beta]
xb = np.array(xb).reshape(-1, 1)

For 4D-Var assimilation we need to provide 2 funtions:
 * observation operator - function that returns observation based on given state
 * evolution step - function that transforms state from one time point to the next one

**TASK 3** write observation operator for Lorenz63 model (Hint: state is an array \[x, y, z, rho, sigma, beta\])

In [ ]:
def lorenz_obs_operator(state):
    return ???

In [ ]:
def lorenz_evol_step(state):
    list(np.ravel(state))
    x, y, z, rho, sigma, beta = state  # Unpack the state vector
    rho, sigma, beta = list(map(lambda x : abs(x), [rho, sigma, beta]))
    dx, dy, dz = lorenz_step([x, y, z], "", rho, sigma, beta)
    return np.array([x + d * dx, y + d * dy, z + d * dz, rho, sigma, beta]).reshape(-1, 1)

**TASK 4** using above functions write a function that generates test observations

In [ ]:
# helper function to generate observations based on evolution function and observation operator
def prepare_obs(state, obs_operator, evolution_function, size=100):
    observations = []
    for i in range(size):
        observations.append(???)
        state = ???
    return observations


In [ ]:
# now we generate truth observations, because assimilation process is quite slow, generate only 200 observations
yobs = np.array(prepare_obs(xb, lorenz_obs_operator, lorenz_evol_step, 200))

In [ ]:
fig = plt.figure()
ax = fig.gca(projection="3d")
ax.plot(yobs[:, 0].flatten(), yobs[:, 1].flatten(), yobs[:, 2].flatten())
plt.draw()
plt.show()

In [ ]:
# now we provide parameters, that assimilation will start from
xb = [1., 1., 1.] + [20., 10., 5.]
xb = np.array(xb).reshape(-1, 1)
error_vector = [0.1, 0.1, 0.1, 100, 100, 100] # diagonal of covariance matrix of each parameter

And now we run 4D-Var data assimilation on generated observations

In [ ]:
result = assimilate(xb, yobs, lorenz_obs_operator, lorenz_evol_step, error_vector=error_vector, verbose=True)

In [ ]:
# generate observations from assimilated parameters
res = np.array(prepare_obs(np.array(result).reshape(-1, 1), lorenz_obs_operator, lorenz_evol_step, 200))

In [ ]:
fig = plt.figure()
ax = fig.gca(projection="3d")
ax.plot(res[:, 0].flatten(), res[:, 1].flatten(), res[:, 2].flatten())
ax.plot(yobs[:, 0].flatten(), yobs[:, 1].flatten(), yobs[:, 2].flatten())
plt.draw()
plt.show()

### SEIR epidemic model

We will try to generate data based on SEIR model and then based on these observations we will use 4D-Var data assimilation to fit parameters.

In our case observations is total number of deaths to given day.

**TASK 5** write observation operator for Lorenz63 model (Hint: state is an array \[s, e, i, r, d, alpha, beta, eps, gamma\])

In [ ]:
def seir_obs_operator(state):
    D = ??
    return D

In [ ]:
# TODO define some random parameters to generate observations
params = [???, ???, ???, ???]
seird = ???, ???, ???, ???, ???]
state = np.array(seird + params).reshape(-1, 1)
days = 200

In [ ]:
# generate observations based on prepared earlier parameters
yobs = prepare_obs(state, seir_obs_operator, seir_step, size=days)

In [ ]:
# now we try to find parameters that will result in matching observations
result = assimilate(seird + [0.01, 0.5, 0.5, 0.5], yobs, seir_obs_operator, seir_step, verbose=True)

In [ ]:
# generate new observations based on result from data assimilation
res = prepare_obs(result, seir_obs_operator, seir_step, size=200)

In [ ]:
plt.plot(range(days), yobs)
plt.plot(range(days), res)

### Assimilation of actual data

Here we try to fir parameters to the actual data concerning COVID-19

In [ ]:
# data can be found in file `covid-deaths.csv` 
from data_assimilation import load_data
yobs = load_data(size = 100, country = 'POL')

In [ ]:
seird = [36000000, 30000, 10000, 80000, 2349]
result = assimilate(seird + [0.01, 0.1, 0.1, 0.1], yobs, seir_obs_operator, seir_step, verbose=True)

In [ ]:
res = prepare_obs(result, seir_obs_operator, seir_step)

In [ ]:
plt.plot(range(100), yobs)
plt.plot(range(100), res)